In [1]:
import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


# Train

In [ ]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0
load data...
Start train...


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


2025-09-03 13:06:40.614498 Epoch [001/002], Step [0001/2184], Loss1: 1.5637 Loss2: 1.4021
Keyboard Interrupt: save model and exit.
save checkpoints successfully!


KeyboardInterrupt: 

: 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [ ]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


Total parameters: 115346414
Trainable parameters: 115346414
Non-trainable parameters: 0


: 

In [ ]:
test(method='fullspatial') 

: 

In [ ]:
# evaluate()

: 

In [ ]:
file_path = "PySODEvalToolkit-master/output/results.xlsx"

: 

In [ ]:
import openpyxl
from openpyxl.styles import PatternFill

def highlight_max_in_metrics(file_path, output_path):
    # Load the workbook and sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    
    # Define the fill style for maximum values
    yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
    
    # Get the maximum row and column indices
    max_row = sheet.max_row
    max_col = sheet.max_column
    
    # Data starts from row 3 (index 3) since rows 1 and 2 are headers
    start_data_row = 3
    
    # Iterate through each column
    for col in range(1, max_col + 1):
        values = []
        # Collect all numeric values in the column
        for row in range(start_data_row, max_row + 1):
            cell = sheet.cell(row=row, column=col)
            try:
                num_val = float(cell.value)
                values.append((num_val, cell))
            except (ValueError, TypeError):
                continue
        
        if not values:
            continue
        
        # Find the maximum value in the column
        max_val = max(values, key=lambda x: x[0])[0]
        
        # Highlight all cells with the maximum value
        for value, cell in values:
            if value == max_val:
                cell.fill = yellow_fill
    
    # Save the modified workbook
    workbook.save(output_path)

# Example usage
input_file = "PySODEvalToolkit-master/output/results.xlsx"
output_file = "PySODEvalToolkit-master/output/results1.xlsx"
highlight_max_in_metrics(input_file, output_file)

: 